In [2]:
import numpy as np
import pydotplus
import pandas as pd
import sklearn.datasets as Datasets
from sklearn import tree
from sklearn import model_selection as cv

In [4]:
#This is a function to convert continuos values to discrete values
def makeLabelled(column):
    mean = column.mean()
    for i in range (0,len(column)):
        column[i] = int(column[i]>=mean) 
    return column

## Modifying Iris Dataset to change it to Labeled Data¶

In [8]:
iris = Datasets.load_iris()
df = pd.DataFrame(iris.data)
df.shape

(150, 4)

In [13]:
X = df.values
Y = iris.target

4
(150,)
[ 5.9  3.   5.1  1.8]


In [14]:
#As iris dataset has continuous values, we will convert them into discrete nunbers
for i in range(0,X.shape[-1]):
    X[:,i] = makeLabelled(X[:,i])

In [15]:
clf = tree.DecisionTreeClassifier(criterion="entropy")
clf.fit(X,Y)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [17]:
dot_data = tree.export_graphviz(clf,out_file= None)
graph = pydotplus.graph_from_dot_data(dot_data)
graph.write_pdf("irisDecisionTree.pdf")

True

## Implementation of Decision Tree

In [19]:
#This function calculates entropy
def entropy(Y):
    classes = set(Y)
    #print("Different Classes are = ",classes)
    value = 0
    for i in classes:
        p = (len(Y[Y==i])/len(Y))
        value = value - (p*np.log2(p))
    return value

In [21]:
#Finding gain ration for a selected feature
def findGainRatio(X,Y,selectedFeature):
    differentLabels = set(X[:,selectedFeature])
    entropyBeforeSplitting = entropy(Y)
    entropyAfterSplitting = 0
    splitInfo = 0
    for i in differentLabels:
        newNodeY = Y[(X[:,selectedFeature] == i)]
        weightOfSamples = (len(newNodeY)/len(Y))
        entropyAfterSplitting = entropyAfterSplitting + (weightOfSamples*entropy(newNodeY))
        splitInfo = splitInfo - (weightOfSamples*np.log2(weightOfSamples))
    #print("Entropy Before Splitting = ",entropyBeforeSplitting)
    #print("Entropy After Splitting = ",entropyAfterSplitting)
    gain = entropyBeforeSplitting - entropyAfterSplitting
    gainRatio = gain#/splitInfo
    #print("gain is = ",gain)
    #print("split info is = ",splitInfo)
    #print("gain Ratio is = ",gainRatio)
    return gainRatio

In [23]:
#This function will count of samples for each class.
def printClassification(Y):
    classes = set(Y)
    for i in classes:
        print("Count of ",i," = ",len(Y[Y==i]))

In [25]:
#Function to create and print Decision tree. This function prints the selected feature at every step and corresponding entropy.
def printSplitsDFS(X,Y,available_features):
    print(" ")
    printClassification(Y)
    print("Current Entropy is = ",entropy(Y))
    if(available_features == 0 or (entropy(Y) == 0)):
        print("Reached leaf Node")
        return
    selectedFeature = 0
    max_value = -float('inf')
    #finding gain ratio for all possible features on which we can split and then choosing the feature with maximum gain.
    for i in range(0,X.shape[-1]):
        value = findGainRatio(X,Y,i)
        if(value >= max_value):
            selectedFeature = i
            max_value = value
    print("Splitting on feature ",selectedFeature)
    #Find all possible unique labels for the selected feature.
    differentLabels = set(X[:,selectedFeature])
    for i in differentLabels:
        newDataSamples = (X[:,selectedFeature] == i)
        newX = X[newDataSamples]
        newY = Y[newDataSamples]
        printSplitsDFS(newX,newY,available_features-1)
    return

In [28]:
printSplitsDFS(X,Y,4)

 
Count of  0  =  50
Count of  1  =  50
Count of  2  =  50
Current Entropy is =  1.58496250072
Splitting on feature  2
 
Count of  0  =  50
Count of  1  =  7
Current Entropy is =  0.537376085338
Splitting on feature  1
 
Count of  0  =  8
Count of  1  =  7
Current Entropy is =  0.996791631982
Splitting on feature  3
 
Count of  0  =  8
Count of  1  =  6
Current Entropy is =  0.985228136034
Splitting on feature  3
 
Count of  0  =  8
Count of  1  =  6
Current Entropy is =  0.985228136034
Reached leaf Node
 
Count of  1  =  1
Current Entropy is =  0.0
Reached leaf Node
 
Count of  0  =  42
Current Entropy is =  0.0
Reached leaf Node
 
Count of  1  =  43
Count of  2  =  50
Current Entropy is =  0.995909413894
Splitting on feature  0
 
Count of  1  =  17
Count of  2  =  6
Current Entropy is =  0.82805572538
Splitting on feature  3
 
Count of  1  =  3
Current Entropy is =  0.0
Reached leaf Node
 
Count of  1  =  14
Count of  2  =  6
Current Entropy is =  0.881290899231
Splitting on feature 

In [29]:
X = np.array([[0,0],[0,1],[1,0],[1,1]])
Y = np.array([0,1,1,0])
printSplitsDFS(X,Y,2)

 
Count of  0  =  2
Count of  1  =  2
Current Entropy is =  1.0
Splitting on feature  1
 
Count of  0  =  1
Count of  1  =  1
Current Entropy is =  1.0
Splitting on feature  0
 
Count of  0  =  1
Current Entropy is =  0.0
Reached leaf Node
 
Count of  1  =  1
Current Entropy is =  0.0
Reached leaf Node
 
Count of  0  =  1
Count of  1  =  1
Current Entropy is =  1.0
Splitting on feature  0
 
Count of  1  =  1
Current Entropy is =  0.0
Reached leaf Node
 
Count of  0  =  1
Current Entropy is =  0.0
Reached leaf Node
